In [1]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
from bs4 import BeautifulSoup
import json
import re

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [46]:
def remove_prefix_from_name(name):
    pattern = r"^(?:[a-z]{2,}\.\s)*(.*)"
    match = re.match(pattern, name, flags=re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return name

print(remove_prefix_from_name('ing. drs. P.F.C.W. van der Broeck'))

P.F.C.W. van der Broeck


TODO: van tree automatisch naar namespaces gaan.

In [47]:
def json_create(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschappen': 'ws',
                  'Gemeenten': 'gm',
                   'Provincie': 'pv'}
    afkorting = afkortingen[organisatie_type]
    
    tree = tree = ET.parse(f"exportOO_{organisatie_type.lower()}.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall('.//p:organisatie', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie_element in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = organisatie_element.find('.//p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces)
        if dc_publisher is not None:
            # hier wordt de afkorting gebruikt bijv. ws = waterschappen
            if dc_publisher.text.rsplit('/')[-1][:2] == afkorting:
                dc_publisher = dc_publisher.text.rsplit('/')[-1]
            else:
                dc_publisher = ''

            # verzamel de naam van de organisatie
            naam_element = organisatie_element.find('p:naam', namespaces)
            if naam_element is not None:
                dc_publisher_name = naam_element.text
            else:
                dc_publisher_name = ''

            # verkrijg de website van deze organisatie evt. om dit later te gebruiken bij webscraping
            website = organisatie_element.find('.//p:contact/p:internetadressen/p:internetadres/p:url', namespaces)
            if website is not None:
                website_txt = website.text


            # en het soort dat de organisatie is (bijv. Waterschap)
            type_element = organisatie_element.find('p:types/p:type', namespaces)
            if type_element is not None:
                Type = type_element.text

            # in het functie_element staan alle verschillende functies
            functie_element = organisatie_element.findall('p:functies/p:functie', namespaces)

            # loop over deze functies
            for functie in functie_element:
                foi_count = 0
                Dict = dict()
                Filtered_Dict = dict()
                DICT = dict()

                # veramel het type functie (bijv. Dijkgraaf)
                function_type = functie.find('p:naam', namespaces)
                if function_type is not None:
                    foaf_function_type = function_type.text

                # vindt alle medewerkers die bij deze functie horen
                medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

                # loop over alle medewerkers om per medewerker gegevens te verzamelen
                for medewerker in medewerker_element:

                    # verzamel de naam in de XML
                    initial_name = medewerker.find('p:naam', namespaces)
                    if initial_name is not None:
                        initial_name_txt = initial_name.text

                        # er staat vaak een prefix als dr. of mew. voor deze wordt verwijderd?
                        name_without_prefix = remove_prefix_from_name(initial_name_txt)
                        name_without_prefix = name_without_prefix.lstrip()
                        
                        initial_name_split = name_without_prefix.split(' ', 1)
                        

                        # bij gemeenten staat er een persoon in als mew. Schouten dus zonder initialen. 
                        if len(initial_name_split) == 2:
                            first_word = initial_name_split[0]
                            if '.' not in first_word and len(first_word) != 1:
                                
                                foaf_firstName = initial_name_split[0]
                                foaf_initials = foaf_firstName[0].upper() + '.'
                            else:
                                foaf_initials = initial_name_split[0]
                                foaf_firstName = ''

                            foaf_lastName = initial_name_split[1]

                        else:
                            foaf_initials = ''
                            foaf_firstName = ''
                            foaf_lastName = initial_name_split[0]


                        # # optioneel vind start datum
                        startDate = medewerker.find('p:startDatum', namespaces)
                        if startDate is not None:
                            foi_startDate = startDate.text
                        else:
                            foi_startDate = ''

                        # optioneel vind telefoonnummer
                        phone_number = medewerker.find('p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)
                        if phone_number is not None:
                            foaf_phone = phone_number.text
                        else:
                            foaf_phone = ''

                        # optioneel vind e-mail adres
                        email = medewerker.find('p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                        if email is not None:
                            foaf_mbox = email.text
                        else:
                            foaf_mbox = ''

                        if organisatie_type == 'Waterschappen':
                            bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {initial_name_txt}, voor {Type.lower()} {dc_publisher_name}"
                        elif organisatie_type == 'Gemeenten':
                            bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {initial_name_txt}, voor {dc_publisher_name}"
                        ### TODO: aanvullen met andere organisatietypes

                        # vul de dictionary in
                        Dict = {
                                'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                                'dc_title': f"{initial_name_txt} - {dc_publisher_name}",
                                'dc_type': foaf_function_type,
                                'dc_description': bereikbaarheidsgegevens,
                                'dc_source': "https://organisaties.overheid.nl/archive/exportOO_waterschappen.xml",
                                'dc_publisher': dc_publisher,
                                'dc_creator': "Ramon Duursma",
                                'foi_retrievedDate': foi_retrievedDate,
                                'dc_date_year': dc_date_year,
                                'dc_publisher_name': dc_publisher_name,
                                'foaf_initials': foaf_initials,
                                'foaf_firstName': foaf_firstName,
                                'foaf_lastName': foaf_lastName,
                                'foaf_name': initial_name_txt,
                                'foaf_mbox': foaf_mbox,
                                'foaf_phone': foaf_phone,
                                'foaf_img': "webscraping required",
                                'foaf_type': Type,
                                'foi_startDate': foi_startDate,
                                'foi_files': [] ,
                            }

                        # Verwijder alle lege strings uit de dict
                        filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                        # en voeg de dictionary toe aan 
                        DICT[foi_count] = filtered_dict

                        foi_count+=1
                    
                # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                            'infobox': {'foi_totalDossiers': len(DICT),
                                         'foi_dossiers': DICT}}



                list_of_dicts.append(final_dict)

    
        

    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)

created_json = json_create('Gemeenten')
